## Importações


In [21]:
#Bibliotecas
import pandas as pd
import os
import json

In [22]:
#Definindo o diretório padrão
caminho = os.chdir('C:\\Users\\anand\\Documents\\BT\\input')

In [23]:
#Salvando os nomes dos arquivos em uma lista para serem usados posteriormente
lista_arquivos = os.listdir(caminho)
nomes_arquivos = []
for arquivo in lista_arquivos:
  if arquivo.endswith('.json'):
    arquivo_semext = arquivo.replace('.json', '')
    nomes_arquivos.append(arquivo_semext)
print(nomes_arquivos)

['TUC20', 'TUC21', 'TUC22', 'TUC23', 'TUC24']


In [24]:
#Carregando os arquivos

todos_arquivos = []
diretorio = r'C:\Users\anand\Documents\BT\\input'
for arquivo in os.listdir(diretorio):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        with open(caminho_arquivo, 'r') as file:
            dados = json.load(file)
            todos_arquivos.append(dados)    
            

## Estudando a estrutura dos arquivos

In [ ]:
#Identificando a estrutura do arquivo
print(todos_arquivos[0].keys())
print(todos_arquivos[0]['objects'].keys())
print(todos_arquivos[0]['objects']['11893575734DC000'].keys())
print(todos_arquivos[0]['objects']['11893575734DC000']['points'].keys())

In [ ]:
#Dicionário provisório com a chave do primeiro arquivo, para identificar a estrutura e extrair os kpis 
dicionario = todos_arquivos[0]['objects']['11893575734DC000']['points'] 
dicionario

In [27]:
#Acrescentando kpis do dicionário em uma lista
kpis = []
for key in dicionario.keys():
  if not key.startswith('Waterfall') and not key.startswith('Calc-Error') and not key.startswith('Calc-LossDeviation') and not key.startswith('Calc-SPEGeneration'): #Removendo kpis desnecessários
    if key not in kpis:
      kpis.append(key)

kpis.append('Waterfall-DeliveredEnergy.5min') #adiciona o único campo "waterfall" necessário
print(kpis)
len(kpis)

['Calc-LossUnavailability.5min', 'Calc-DeltaOverIrrad.5min', 'Calc-DeltaIrrad.5min', 'Calc-DeltaTemp.5min', 'Calc-DeltaAlbedo.5min', 'Calc-ExpectedDC.5min', 'Calc-LossSoiling.5min', 'Calc-LossShading.5min', 'Calc-LossStringPerf.5min', 'Calc-LossEfficiency.5min', 'Calc-EnergyDC.5min', 'Calc-LossClipping.5min', 'Calc-LossClippingTemp.5min', 'Calc-LossGridLimitation.5min', 'Calc-LossCurtailment.5min', 'Calc-LossInvToSPE.5min', 'Waterfall-DeliveredEnergy.5min']


17

## Criando os dataframes

### Dataframes com valores separados por SPE e datas

In [28]:
#Criando um dataframe para cada arquivo .json importado
num = 0
for arquivo in todos_arquivos:
  obj_key = list(arquivo['objects'].keys()) #Identificando a object key de cada arquivo
  print(obj_key)
  dicionario = arquivo['objects'][obj_key[0]]['points'] #Criando um dicionário para cada arquivo a partir da object key identificada
  kpi_1 = kpis[0]
  df1 = pd.DataFrame(dicionario[kpi_1][0]['timeSeries']) #Criando o primeiro dataframe com os kpis, onde será mantida a coluna de tempo
  df1 = df1.drop(columns=['t_local','q'])
  df1 = df1.rename(columns={'t':'Time', 'v': kpis[0]})
  df1 = df1.set_index('Time') #Definindo a coluna 'Time' como index
  for n in range (1, len(kpis)): #Criando os demais dataframes, com somente a coluna 'v'
    if kpis[n] != kpis[0]:
      kpi_n = kpis[n]
      df2 = pd.DataFrame(dicionario[kpi_n][0]['timeSeries'])
      df2 = df2.drop(columns=['t_local','q'])
      df2 = df2.rename(columns={'t':'Time', 'v': kpis[n]})
      df2 = df2.set_index('Time') #Definindo a coluna 'Time' como index
      df1 = pd.merge(df1,df2, left_index=True, right_index=True, how='inner') #Unindo todos os dataframes ao primeiro, a partir do index
  df1.head()
  df1['Meta'] = df1['Calc-ExpectedDC.5min'] - (df1['Calc-DeltaAlbedo.5min'] + df1['Calc-DeltaTemp.5min'] + df1['Calc-DeltaIrrad.5min'])
  df1['Date'] = df1.index
  df1['Date'] = pd.to_datetime(df1['Date'], unit='ms')
  df1 = df1.rename(columns={'Calc-EnergyDC.5min':'Calc-EnergyAC.5min'})
  df_name = nomes_arquivos[num] #Nomeando os arquivos a partir da lista com nome dos .jsons importados
  num+=1
  #Renomeando as colunas
  df1.columns = [
      coluna.replace('Calc-', '').replace('.5min', '').replace('Waterfall-', '')
      for coluna in df1.columns
  ]
  #Salvando os dataframes
  df1.to_excel('C:\\Users\\anand\\Documents\\BT\\output\\consol_SPE_excel\\' + 'consolidated_' + df_name +'.xlsx', engine ='xlsxwriter') 
  print(f'{df_name} salvo!')
 

['11893575734DC000']
TUC20 salvo!
['11893575734DC001']
TUC21 salvo!
['11893575734DC002']
TUC22 salvo!
['11893575734DC003']
TUC23 salvo!
['11893575734DC004']
TUC24 salvo!


In [29]:
df1.head()

,LossUnavailability,DeltaOverIrrad,DeltaIrrad,DeltaTemp,DeltaAlbedo,ExpectedDC,LossSoiling,LossShading,LossStringPerf,LossEfficiency,EnergyAC,LossClipping,LossClippingTemp,LossGridLimitation,LossCurtailment,LossInvToSPE,DeliveredEnergy,Meta,Date
Time,,,,,,,,,,,,,,,,,,,
1730430000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2024-11-01 03:00:00
1730430300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2024-11-01 03:05:00
1730430600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2024-11-01 03:10:00
1730430900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2024-11-01 03:15:00
1730431200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2024-11-01 03:20:00


In [30]:
#Salvando os nomes dos arquivos em uma lista para serem usados posteriormente
caminho_dataframes = os.chdir('C:\\Users\\anand\\Documents\\BT\\output\\consol_SPE_excel')
lista_dataframes = os.listdir(caminho_dataframes)
nomes_dataframes = []
for dataframe in lista_dataframes:
  if dataframe.endswith('.xlsx'):
    dataframe_semext = dataframe.replace('consolidated_', '')
    dataframe_semext = dataframe_semext.replace('.xlsx', '')
    nomes_dataframes.append(dataframe_semext)
print(nomes_dataframes)

['TUC20', 'TUC21', 'TUC22', 'TUC23', 'TUC24']


### Dataframe único com valores acumulados de todas SPEs e separados por datas

In [36]:
df_diario = pd.DataFrame()
num = 0
for excel in os.listdir(caminho_dataframes):
    print(excel)
    caminho_excel = os.path.join(caminho_dataframes, excel)
    if excel.endswith('.xlsx'):
        dataframe = pd.read_excel(caminho_excel, sheet_name=None)
        dataframe = dataframe.drop(columns='Date')
        df_diario_name = nomes_arquivos[num]  
        dataframe['SPE'] = df_diario_name    
    if df_diario.empty:
        df_diario = dataframe
    else:
        df_diario = pd.concat([df_diario, dataframe])
    num +=1

print("DataFrame acumulado:")
df_diario.head()
df_diario['Date'] = df_diario['Time']
df_diario['Date'] = pd.to_datetime(df_diario['Date'], unit='ms')
df_diario['Deviation'] = df_diario['DeliveredEnergy'] + df_diario['LossClipping'] + df_diario['LossClippingTemp'] + df_diario['LossGridLimitation'] + df_diario['LossCurtailment'] + df_diario['LossUnavailability'] - df_diario['EnergyAC']
df_diario['LossEfficiency'] = -df_diario['EnergyAC'] + df_diario['ExpectedDC'] - df_diario['LossSoiling'] - df_diario['LossShading'] - df_diario['LossStringPerf'] 
df_diario['Resource'] = df_diario['DeltaIrrad'] + df_diario['DeltaTemp'] + df_diario['DeltaAlbedo']
df_diario['LossDC'] = df_diario['LossSoiling'] + df_diario['LossShading'] + df_diario['LossStringPerf'] + df_diario['LossEfficiency']
df_diario['LossAC'] = df_diario['LossClipping'] + df_diario['LossClippingTemp'] + df_diario['LossGridLimitation'] + df_diario['LossCurtailment'] + df_diario['LossUnavailability']
df_diario['NoCurtailment'] = df_diario['DeliveredEnergy'] + df_diario['LossCurtailment']
df_diario.to_excel('C:\\Users\\anand\\Documents\\BT\\output\\'+ 'diario'+'.xlsx', engine ='xlsxwriter') 

consolidated_TUC20.xlsx
consolidated_TUC21.xlsx
consolidated_TUC22.xlsx
consolidated_TUC23.xlsx
consolidated_TUC24.xlsx
DataFrame acumulado:


In [37]:
print(df_diario['LossDC'].sum())
print((df_diario['LossSoiling'] + df_diario['LossShading'] + df_diario['LossStringPerf'] + df_diario['LossEfficiency']).sum())

419577.1026332099
419577.1026332099
